In [14]:
# Dependencies
import pandas as pd
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import DataFrameLoader
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

import chromadb

# Create environment variable for OpenAI API key
from config import openai_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key


In [2]:
# Initialize Chroma client
chroma_client = chromadb.Client()

Using embedded DuckDB without persistence: data will be transient


In [3]:
# Create a Chroma collection
collection = chroma_client.create_collection(name="nfl-prospects-1")

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [4]:
# Define the data
data = pd.read_csv('data/dataset.csv')

In [5]:
# Create the loader specify the column containing the text
loader = DataFrameLoader(data, page_content_column="Player Bio")

In [6]:
# Load the data
docs = loader.load()

In [8]:
# Define the embeddings
embeddings = OpenAIEmbeddings()

In [10]:
# Create the Chroma database
db = Chroma.from_documents(docs, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [11]:
persist_directory = 'db'

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=docs, embedding=embedding, persist_directory=persist_directory)


Using embedded DuckDB with persistence: data will be stored in: db


In [12]:
vectordb.persist()
vectordb = None

In [13]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: db


In [16]:
llm = OpenAI(temperature=0.7)
chain = load_qa_chain(llm, chain_type="stuff")

In [17]:
retriever = db.as_retriever(search_type="mmr")

In [18]:
query = "What do you think of Justin Fields?"

In [19]:
chain.run(input_documents=retriever, question=query)


AttributeError: 'tuple' object has no attribute 'page_content'

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal == 2021.2.3, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
numba 0.54.1 requires numpy < 1.21, >= 1.17, but you have numpy 1.22.4 which is incompatible.
federal-reserve-python-api 0.1.0 requires requests == 2.24.0, but you have requests 2.28.2 which is incompatible.
